In [938]:
import numpy as np
from numpy.random import choice
import random
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
##standing wave solution of Fisher wave
def standing_wave(y0,x,D,rw):
    w = y0[0]      ##initial value for wave profile at x =0, i.e. w(x=0)
    z = y0[1]      ##initial value for rate of change of profile w.r.t. position x , at x=0 i.e. dw/dx(x=0)
    dwdx = z
    dzdx =(-2*((rw*D)**.5)*dwdx -w*rw*(1-w))/D
    
    return [dwdx,dzdx]

In [1110]:
def rand_neighbors(demes):
    ind_1 = np.random.choice(demes)
    left = demes[:ind_1][-1:]
    right = demes[ind_1:][1:2]
    neighb = np.append(left,right).flatten()
    ind_2=choice(neighb)
    neigh = [ind_1,ind_2]
    neigh.sort()
    return np.array(neigh)


def counts_to_cells(counts,n_allele):
    cells = np.repeat(np.arange(n_allele+1),counts)
    return cells
    
def cells_to_counts(cells,n_allele):
    counts = np.bincount(cells, minlength=n_allele+1)
    return counts
    
    
def migration(cells,K):
    cells_1 = cells[0]
    cells_2 = cells[1]
    pick_ind=choice(np.arange(K),2,replace= True)
    picks = np.array([np.arange(K) == pick_ind[i] for i in [0,1]])
    keep =  ~picks
    cells_1 = np.append(cells_1[keep[0]], cells_2[picks[1]])
    cells_2 = np.append(cells_2[keep[1]], cells_1[picks[0]])
    return np.array([cells_1,cells_2])


def duplication(cells,K,P):
    pick_ind=choice(np.arange(K),2,replace= False)
    picks = np.array([np.arange(K) == pick_ind[i] for i in [0,1]])
    r= np.random.random()
    if P[tuple(cells[pick_ind])]> r:
        cells[pick_ind[1]] =cells[pick_ind[0]]
    return cells
    
    
def mutation(cells,mu,K,n_allele):
    pick_ind=choice(np.arange(K))
    r= np.random.random()
    if mu>r:
        if cells[pick_ind] != n_allele and cells[pick_ind] !=0:
            cells[pick_ind] = cells[pick_ind] +1
    return cells
    

In [1143]:
def update(L,L_empty,P,K,n_allele,r,alpha,mu):

        demes = np.arange(len(L))
        #migration
        neighb = rand_neighbors(demes)
        cells = np.array(list(map(counts_to_cells, L[neighb],2*[n_allele] )))
        cells = migration(cells,K)
        counts =  np.array(list(map(cells_to_counts, cells,2*[n_allele] )))
        L[neighb] = counts

        #duplication
        dup_deme = choice(demes)
        cells = counts_to_cells(L[dup_deme],n_allele)
        cells = duplication(cells,K,P)
        counts = cells_to_counts(cells, n_allele)
        L[dup_deme] = counts

        ##mutation
        mut_deme = choice(demes)
        cells = counts_to_cells(L[mut_deme],n_allele)
        cells = mutation(cells,mu,K,n_allele)
        counts = cells_to_counts(cells, n_allele)
        L[mut_deme] = counts


        ##recenter frame
        if L[-1][0]!=K:
            L=np.append(L,[L_empty],axis=0)
        L=L[(L[:,0]>0)]
        return L


In [1200]:
n_gen=100
K=1000
n_allele=1
r=.1
alpha=2
mu=.001
L_h = run_stepping_stone(n_gen,K,n_allele,r,alpha,mu)


In [1182]:
def run_stepping_stone(n_gen,K,n_allele,r,alpha,mu):
    func_args = [K,n_allele,r,alpha,mu]
    ##initialize probability matrix
    P = np.ones((n_allele+1,n_allele+1))
    P[0,1:] = 1-r*(alpha**np.arange(n_allele))
    
    stand = odeint(standing_wave,[1,-r**.5],np.arange(70),args=(1,r))[:,0]
    w_0 = (K*stand).astype(int)
    w_0 = w_0[w_0>1]
    
    ##initialize array
    L_empty= np.append([K],np.zeros(n_allele,dtype=int))
    L0 = np.zeros((len(w_0),n_allele+1),dtype=int)
    L0[:,1] = w_0
    L0[:,0] = K-w_0
    L= np.append(L0,[L_empty],axis=0)
    L_history=[L]
    #begin evolution
    for t in range(n_gen):
        for dt in range(K):
            L_history.append(L)
            L = update(L,L_empty,P,*func_args)

    return L_history


In [1220]:
def fix_time_2allele(K,r,alpha,mu,thresh):
    func_args = [K,n_allele,r,alpha,mu]
    ##initialize probability matrix
    P = np.ones((n_allele+1,n_allele+1))
    P[0,1:] = 1-r*(alpha**np.arange(n_allele))
    
    stand = odeint(standing_wave,[1,-r**.5],np.arange(70),args=(1,r))[:,0]
    w_0 = (K*stand).astype(int)
    w_0 = w_0[w_0>1]
    
    ##initialize array
    L_empty= np.append([K],np.zeros(n_allele,dtype=int))
    L0 = np.zeros((len(w_0),n_allele+1),dtype=int)
    L0[:,1] = w_0
    L0[:,0] = K-w_0
    L= np.append(L0,[L_empty],axis=0)
 

    #begin evolution
    fixed=False
    fixed_hist=[]
    est_hist=[]
    t = 0
    while not fixed:
        L = update(L,L_empty,P,*func_args)
        fix_bools = L[:,1]< int(thresh*K)
        est_bools = L[:,2]> int(1/(r*alpha))
        fixed= all(fix_bools)
        if any(fix_bools):
            fixed_hist.append(fix_bools)
            
        if any(est_bools):
            est_hist.append(est_bools)
        t+=1

    return fixed_hist,est_hist,L

In [1221]:
K=1000
n_allele=2
r=.1
alpha=5
mu=.001

fix,est,L = fix_time_2allele(K,r,alpha,mu,thresh=.001)

In [1226]:
fix[-1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [901]:
K=100
n_allele=2
r=.1
alpha=2
mu=.001
thresh=.2

func_args = [K,n_allele,r,alpha,mu]
##initialize probability matrix
P = np.ones((n_allele+1,n_allele+1))
P[0,1:] = 1-r*alpha**np.arange(n_allele)
##initialize array
L_empty= np.append([K],np.zeros(n_allele,dtype=int))
L0 = np.zeros(n_allele+1,dtype=int)
L0[1] = K
L= np.append([L0],[L_empty],axis=0)


#begin evolution
fixed=False
fixed_hist=[]
t = 0
while not fixed:
    L = update(L,L_empty,P,*func_args)
    fix_bools = L[:,1]< int(thresh*K)
    est_bools = L[:,2]< int(1/(r*alpha))
    fixed= all(fix_bools)
    if any(fix_bools):
        fixed_hist.append(fix_bools)
    t+=1



KeyboardInterrupt: 

In [1207]:
fix[-1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])